In [ ]:
!conda install faiss-gpu cudatoolkit=9.0 -c pytorch-y 
#Fast nearest neighbors will help us a lot :)

In [1]:
import matplotlib.pyplot as plt
from third_party.testdataset import configdataset
from third_party.evaluate import compute_map_and_print

from fastai import *
from fastai.vision import *
import pandas as pd
from arch import RingGeMNet, GeMNet
from losses import RingLoss
from utils import extract_vectors_batched


In [2]:
#I suppose, that competition dataset is downloaded to COMP_DATA_DIR
#and the list of images are in index_image_list_fullpath.txt
COMP_DATA_DIR = '/mnt/fry2/users/datasets/landmarkscvprw18/retrieval'
df = pd.read_csv(os.path.join(COMP_DATA_DIR, 'index_image_list_fullpath.txt'),
                  usecols=[0], names=['Image'])
df.head()


,Image
0,/mnt/fry2/users/datasets/landmarkscvprw18/retr...
1,/mnt/fry2/users/datasets/landmarkscvprw18/retr...
2,/mnt/fry2/users/datasets/landmarkscvprw18/retr...
3,/mnt/fry2/users/datasets/landmarkscvprw18/retr...
4,/mnt/fry2/users/datasets/landmarkscvprw18/retr...


In [3]:
class ImageListAbsPath(ImageList):
    def open(self, fn:PathOrStr)->Image:
        return open_image(fn.replace('./',''))
tfms = get_transforms(do_flip=False)
tfms = (tfms[1],tfms[1]) 
DO_FULL_SIZE = False 
# Extracting features from full-size images would take 15 hours. You probably want to do this 
#for real submission, not for sample. So lets do it on 256x256 images
if DO_FULL_SIZE:
    BS=1
    NUM_WORKERS=8
    data = (ImageListAbsPath.from_df(df,path='', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.NO)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           ) 
    data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(data.train_ds)
    data.train_dl.dl.batch_sampler.drop_last = False
if not DO_FULL_SIZE:
    BS=64
    NUM_WORKERS=8
    data = (ImageListAbsPath.from_df(df,path='', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.SQUISH, size=192)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           ) 
    data.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(data.train_ds)
    data.train_dl.dl.batch_sampler.drop_last = False
    #index_features = extract_vectors_batched(data,InferenceNet, BS)
#torch.save(index_features, 'densenet121_pretrained_256px_index_feats.pth')


In [4]:
#Now we will load training network and transfer its weights to the inference net
TRAIN_CLASSES=780 #that is hardcoded from number of classes in training dataset
learn = Learner(data, RingGeMNet(models.densenet121(pretrained=True), TRAIN_CLASSES),
                   metrics=[accuracy],
                   loss_func=nn.CrossEntropyLoss())

InferenceNet =  GeMNet(models.densenet121())
InferenceNet.cnn.load_state_dict(learn.model.cnn.state_dict())


/home.dokt/mishkdmy/anaconda3/envs/fastai1/lib/python3.7/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [5]:
#This will take 30 min on Titan X
if not DO_FULL_SIZE:
    qdf = pd.read_csv(os.path.join(COMP_DATA_DIR, 'test_image_list_fullpath.txt'),
                  usecols=[0], names=['Image'])
    qdf.head()
    BS=64
    NUM_WORKERS=6
    qdata = (ImageListAbsPath.from_df(qdf,path='', cols=['Image'])
            .split_none()
            .label_const()
            .transform(tfms, resize_method=ResizeMethod.SQUISH, size=256)
            .databunch(bs=BS, num_workers=NUM_WORKERS)
            .normalize(imagenet_stats)
           ) 
    qdata.train_dl.dl.batch_sampler.sampler = torch.utils.data.SequentialSampler(qdata.train_ds)
    qdata.train_dl.dl.batch_sampler.drop_last = False
    #query_features = extract_vectors_batched(qdata,InferenceNet, BS)
#torch.save(query_features, 'densenet121_pretrained_256px_query_feats.pth')

In [6]:

query_features = torch.load('densenet121_pretrained_256px_query_feats.pth').numpy()
index_features = np.zeros((len(data.train_ds),1024)).astype(np.float32)

In [7]:
#Now lets do the nearest neighbor search and create the submission
import faiss

query_fnames = [x.split('/')[-1].replace('.jpg','') for x in qdf.Image.tolist()]
index_fnames = [x.split('/')[-1].replace('.jpg','') for x in df.Image.tolist()]


In [8]:
from fastprogress import master_bar, progress_bar

def get_idxs_and_dists(query_features, index_features, BS = 32):
    flat_config = faiss.GpuIndexFlatConfig()
    flat_config.device = 0
    res = faiss.StandardGpuResources()
    co = faiss.GpuClonerOptions()
    FEAT_DIM = index_features.shape[1]
    cpu_index = faiss.IndexFlatL2(FEAT_DIM)
    cpu_index.add(index_features)
    index = faiss.index_cpu_to_gpu(res, 0, cpu_index, co)
    out_dists = np.zeros((len(query_features), 100), dtype=np.float32)
    out_idxs = np.zeros((len(query_features), 100), dtype=np.int32)
    NUM_QUERY = len (query_features)
    for ind in progress_bar(range(0, len(query_features), BS)):
        fin = ind+BS
        if fin > NUM_QUERY:
            fin = NUM_QUERY
        q_descs = query_features[ind:fin]
        D, I = index.search(q_descs, 100)
        out_dists[ind:fin] = D
        out_idxs[ind:fin] = I
    return out_idxs, out_dists

def create_submission_from_features(query_features,
                                    index_features,
                                    fname,
                                    query_fnames,
                                    index_fnames):
    out_idxs, out_dists = get_idxs_and_dists(query_features, index_features, BS = 32)
    print (f'Writing {fname}')
    with open(fname, 'w') as f:
        f.write('id,images\n')
        for i in progress_bar(range(len(query_fnames))):
            ids = [index_fnames[x] for x in out_idxs[i]]
            f.write(query_fnames[i] + ',' + ' '.join(ids)+'\n')
    print('Done!')
    return
create_submission_from_features(query_features, index_features, 'test_submission.csv',
                                   query_fnames, index_fnames)
        
        
    



Writing test_submission.csv


Done!
